In [1]:
# Standard Libraries
import os
import sys
import warnings
from datetime import datetime
from time import time
import unicodedata

# Data Manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from os import listdir
from joblib import dump
from joblib import load
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import auc

# Détermine si vous êtes sur Google Colab
is_colab = 'google.colab' in sys.modules

# Détermine si vous êtes sur Kaggle
is_kaggle = '/kaggle' in os.getcwd()

# Chemin par défaut
path = None

if is_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    path = '/content/drive/MyDrive/Python/OCRP/Projet07/data/credit-default-risk'
    output_dir = '/content/drive/MyDrive/Python/OCRP/Projet07/working/graphs'
    os.makedirs(output_dir, exist_ok=True)
    print("\nTu es sur Google Colab")
elif is_kaggle:
    path = "/kaggle/input/credit-risk"
    output_dir = '/kaggle/working/graphs'
    os.makedirs(output_dir, exist_ok=True)
    print("\nTu es sur Kaggle")
else:
    path = "~/Documents/Python/OCR/Projet07/data/credit-risk"
    output_dir = "~/Documents/Python/OCR/Projet07/working/graphs"
    os.makedirs(output_dir, exist_ok=True)
    print("\nTu es en local")

# Utilisez le chemin d'accès sélectionné pour accéder à vos données
if path is not None:
    print("Chemin d'accès aux données:", path)
else:
    print("Impossible de déterminer l'environnement.")
# Chemin d'accès aux données
save_path = "/kaggle/working/"


Mounted at /content/drive

Tu es sur Google Colab
Chemin d'accès aux données: /content/drive/MyDrive/Python/OCRP/Projet07/data/credit-default-risk


## Chargement des données

In [2]:
# Charger le DataFrame avec pandas
path = '/content/drive/MyDrive/Python/OCRP/Projet07/working/data/'
df_cleaned_filtered = pd.read_csv(os.path.join(path, 'df_64_features.csv'))
print("Dimenssion du dataset groupé:", df_cleaned_filtered.shape)

Dimenssion du dataset groupé: (356251, 66)


In [3]:
import pandas as pd
import re
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from contextlib import contextmanager
import time

@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

def one_hot_encoder(df, nan_as_category=True):
    """
    Encodage one-hot pour les colonnes catégorielles et booléennes avec get_dummies.

    Paramètres :
    - df : pandas.DataFrame
        Le DataFrame contenant les données.
    - nan_as_category : bool, facultatif, default=True
        Indique si les NaN doivent être traités comme une catégorie à part.

    Retour :
    - df : pandas.DataFrame
        Le DataFrame avec les colonnes encodées en one-hot.
    - new_columns : list
        Liste des nouveaux noms de colonnes ajoutées lors de l'encodage.
    """

    original_columns = list(df.columns)

    # Identifier les colonnes catégorielles (type object) et booléennes
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    bool_columns = [col for col in df.columns if df[col].dtype == 'bool']

    # Combiner les colonnes catégorielles et booléennes pour l'encodage one-hot
    columns_to_encode = categorical_columns + bool_columns

    # Appliquer l'encodage one-hot
    df = pd.get_dummies(df, columns=columns_to_encode, dummy_na=nan_as_category)

    # Identifier les nouvelles colonnes créées par l'encodage one-hot
    new_columns = [c for c in df.columns if c not in original_columns]

    return df, new_columns

# Charger le DataFrame avec pandas
with timer("One-hot encoding"):
    df_encoded, new_cols = one_hot_encoder(df_cleaned_filtered)

with timer("Nettoyage des noms de colonnes"):
    df_encoded = df_encoded.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '_', x.upper()))

with timer("Séparation des colonnes cibles et d'identification"):
    features = df_encoded.drop(['TARGET', 'SK_ID_CURR'], axis=1)  # Exclure 'TARGET' et 'SK_ID_CURR'
    target = df_encoded['TARGET']  # Colonne cible
    ids = df_encoded['SK_ID_CURR']  # Colonne d'identification

# Vérifier et supprimer les valeurs manquantes dans la colonne cible
with timer("Traitement des valeurs manquantes dans la colonne cible"):
    target = target.dropna()
    features = features.loc[target.index]
    ids = ids.loc[target.index]

with timer("Imputation des valeurs manquantes avec la médiane"):
    imputer = SimpleImputer(strategy='median')
    features_imputed = pd.DataFrame(imputer.fit_transform(features), columns=features.columns, index=features.index)

with timer("Réintégration des colonnes cibles et d'identification"):
    df_imputed = pd.concat([features_imputed, target, ids], axis=1)

with timer("Split stratifié des données"):
    X = df_imputed.drop(['TARGET'], axis=1)  # Features sans la colonne cible
    y = df_imputed['TARGET']  # Colonne cible
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Afficher les dimensions des ensembles d'entraînement et de test
print("Dimensions de l'ensemble d'entraînement (X_train) :", X_train.shape)
print("Dimensions de l'ensemble de test (X_test) :", X_test.shape)
print("Dimensions de l'ensemble d'entraînement (y_train) :", y_train.shape)
print("Dimensions de l'ensemble de test (y_test) :", y_test.shape)


One-hot encoding - done in 0s
Nettoyage des noms de colonnes - done in 0s
Séparation des colonnes cibles et d'identification - done in 0s
Traitement des valeurs manquantes dans la colonne cible - done in 0s
Imputation des valeurs manquantes avec la médiane - done in 5s
Réintégration des colonnes cibles et d'identification - done in 0s
Split stratifié des données - done in 0s
Dimensions de l'ensemble d'entraînement (X_train) : (246005, 85)
Dimensions de l'ensemble de test (X_test) : (61502, 85)
Dimensions de l'ensemble d'entraînement (y_train) : (246005,)
Dimensions de l'ensemble de test (y_test) : (61502,)



## Logistique Regression

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, precision_score, f1_score, log_loss
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import mlflow
import mlflow.sklearn

# Configurer MLFlow
mlflow.set_experiment("scoring_model_experiment")


# Définir la pipeline avec SMOTE et un modèle de régression logistique
model = ImbPipeline([
    ('sampling', SMOTE()),
    ('classification', LogisticRegression(solver='liblinear'))
])

# Définir les hyperparamètres à optimiser
param_grid = {
    'classification__C': [0.1, 1, 10, 100],
    'classification__penalty': ['l1', 'l2']
}

# Configurer GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='roc_auc', n_jobs=-1, verbose=1)

# Logger avec MLFlow
with mlflow.start_run():
    # Entraîner le modèle avec GridSearchCV
    grid_search.fit(X_train, y_train)

    # Obtenir les meilleurs paramètres
    best_params = grid_search.best_params_
    print("Best parameters found: ", best_params)

    # Prédictions sur l'ensemble de test
    y_pred = grid_search.predict(X_test)
    y_pred_proba = grid_search.predict_proba(X_test)[:, 1]

    # Calculer les métriques
    auc = roc_auc_score(y_test, y_pred_proba)
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    logloss = log_loss(y_test, y_pred_proba)

    print(f"AUC: {auc}")
    print(f"Accuracy: {accuracy}")
    print(f"Recall: {recall}")
    print(f"Precision: {precision}")
    print(f"F1 Score: {f1}")
    print(f"Log Loss: {logloss}")

    # Logger les métriques et les paramètres dans MLFlow
    mlflow.log_params(best_params)
    mlflow.log_metric("AUC", auc)
    mlflow.log_metric("Accuracy", accuracy)
    mlflow.log_metric("Recall", recall)
    mlflow.log_metric("Precision", precision)
    mlflow.log_metric("F1 Score", f1)
    mlflow.log_metric("Log Loss", logloss)

    # Logger le modèle
    mlflow.sklearn.log_model(grid_search.best_estimator_, "model")

# Afficher les premières lignes des ensembles d'entraînement et de test
print("Dimensions de l'ensemble d'entraînement (X_train) :", X_train.shape)
print("Dimensions de l'ensemble de test (X_test) :", X_test.shape)
print("Dimensions de l'ensemble d'entraînement (y_train) :", y_train.shape)
print("Dimensions de l'ensemble de test (y_test) :", y_test.shape)


In [2]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.6/505.6 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.

In [3]:
!pip install pyngrok

In [ ]:
!mlflow ui